In [ ]:
import numpy as np

# True covariance matrix
true_Sigma = np.array([[1, 0], [0, 2]])

# List of sample sizes
sample_sizes = [10, 100, 1000]

# For each n, generate samples and compute MLE estimate
for n in sample_sizes:
    # Step 1: Generate samples
    y = np.random.multivariate_normal(mean=[0, 0], cov=true_Sigma, size=n)

    # Step 2: Compute MLE of covariance
    mle_Sigma = (y.T @ y) / n

    print(f"n = {n}")
    print("MLE Estimate of Sigma:")
    print(mle_Sigma)
    print("-" * 30)

n = 10
MLE Estimate of Sigma:
[[0.40342017 0.06384728]
 [0.06384728 1.47652275]]
------------------------------
n = 100
MLE Estimate of Sigma:
[[1.34248235 0.06127777]
 [0.06127777 1.84201507]]
------------------------------
n = 1000
MLE Estimate of Sigma:
[[0.98637743 0.04242716]
 [0.04242716 1.91840893]]
------------------------------


In [ ]:
import numpy as np

def bayesian_cov_estimation(n, true_cov, delta_0, nu_0, seed=42):
    np.random.seed(seed)
    d = true_cov.shape[0]

    # Generate n samples from N(0, true_cov)
    y = np.random.multivariate_normal(mean=[0, 0], cov=true_cov, size=n)

    # Calculate S = sum(y_i * y_i^T)
    S = np.sum([np.outer(yi, yi) for yi in y], axis=0)

    # Posterior parameters
    nu_n = nu_0 + n
    delta_n = delta_0 + S

    # Posterior mean of inverse Wishart
    cov_posterior_mean = delta_n / (nu_n - d - 1)

    return cov_posterior_mean

# Parameters
true_cov = np.array([[1, 0], [0, 2]])
delta_0 = np.array([[4, 0], [0, 5]])
nu_0 = 5

# Run for different n values
for n in [10, 100, 1000]:
    est = bayesian_cov_estimation(n, true_cov, delta_0, nu_0)
    print(f"n = {n}")
    print("Bayesian Estimate of Sigma:")
    print(np.round(est, 4))
    print("-" * 30)

n = 10
Bayesian Estimate of Sigma:
[[1.1303 0.4262]
 [0.4262 1.8391]]
------------------------------
n = 100
Bayesian Estimate of Sigma:
[[1.0087 0.0325]
 [0.0325 1.4986]]
------------------------------
n = 1000
Bayesian Estimate of Sigma:
[[1.0343 0.0062]
 [0.0062 1.851 ]]
------------------------------


In [ ]:
import numpy as np

def generate_data(n, true_cov):
    return np.random.multivariate_normal(mean=[0, 0], cov=true_cov, size=n)

def bayesian_estimate_noninformative(data, nu0, Delta0):
    n, d = data.shape
    S = data.T @ data  # sufficient statistic
    nu_n = nu0 + n
    Delta_n = Delta0 + S
    Sigma_est = Delta_n / (nu_n - d - 1)
    return Sigma_est

np.random.seed(42)
true_cov = np.array([[1, 0], [0, 2]])
Delta0_jeffreys = np.zeros((2, 2))
n_values = [10, 100, 1000]

# Results
print("Results using Jeffreys prior (|Σ|^{-2}):")
for n in n_values:
    data = generate_data(n, true_cov)
    Sigma_jeff = bayesian_estimate_noninformative(data, nu0=3, Delta0=Delta0_jeffreys)
    print(f"n = {n}")
    print("Estimate of Sigma:")
    print(np.round(Sigma_jeff, 4))
    print("-" * 30)

print("\nResults using Independence-Jeffreys prior (|Σ|^{-3/2}):")
for n in n_values:
    data = generate_data(n, true_cov)
    Sigma_indep_jeff = bayesian_estimate_noninformative(data, nu0=2.5, Delta0=Delta0_jeffreys)
    print(f"n = {n}")
    print("Estimate of Sigma:")
    print(np.round(Sigma_indep_jeff, 4))
    print("-" * 30)

Results using Jeffreys prior (|Σ|^{-2}):
n = 10
Estimate of Sigma:
[[0.9564 0.5115]
 [0.5115 1.7069]]
------------------------------
n = 100
Estimate of Sigma:
[[1.0909 0.0677]
 [0.0677 1.4259]]
------------------------------
n = 1000
Estimate of Sigma:
[[ 1.0031 -0.0091]
 [-0.0091  1.9243]]
------------------------------

Results using Independence-Jeffreys prior (|Σ|^{-3/2}):
n = 10
Estimate of Sigma:
[[ 0.7274 -1.0378]
 [-1.0378  2.6497]]
------------------------------
n = 100
Estimate of Sigma:
[[0.8904 0.1437]
 [0.1437 1.8624]]
------------------------------
n = 1000
Estimate of Sigma:
[[1.0381 0.0243]
 [0.0243 2.0275]]
------------------------------


In [ ]:
import numpy as np
from scipy.stats import invwishart

def generate_data(n, true_cov):
    return np.random.multivariate_normal(mean=[0, 0], cov=true_cov, size=n)

def log_likelihood(Sigma, data):
    n, d = data.shape
    sign, logdet = np.linalg.slogdet(Sigma)
    if sign <= 0:
        return -np.inf  # Invalid covariance matrix

    inv_sigma = np.linalg.inv(Sigma)
    quad_form = np.sum([y.T @ inv_sigma @ y for y in data])
    return (-n / 2) * logdet - 0.5 * quad_form

def monte_carlo_bayes(data, nu0, delta0, m):
    d = delta0.shape[0]
    samples = invwishart.rvs(df=nu0, scale=delta0, size=m)
    if m == 1:
        samples = [samples]

    log_ws = np.array([log_likelihood(Sigma, data) for Sigma in samples])

    # log-sum-exp trick for numerical stability
    max_log_w = np.max(log_ws)
    stable_ws = np.exp(log_ws - max_log_w)
    weights = stable_ws / np.sum(stable_ws)

    # Weighted average of the samples
    weighted_sum = sum(w * S for w, S in zip(weights, samples))
    return weighted_sum

# Parameters
true_cov = np.array([[1, 0], [0, 2]])
n_values = [10, 100, 1000]
m_values = [10**3, 10**4, 10**5]

# Priors
priors = {
    "Prior A": {"nu0": 5, "delta0": np.array([[4, 0], [0, 5]])},
    "Prior B": {"nu0": 5, "delta0": np.array([[2, 0], [0, 4]])}
}

# Running the simulations
results = {}

for prior_name, prior_params in priors.items():
    results[prior_name] = {}
    for n in n_values:
        data = generate_data(n, true_cov)
        results[prior_name][n] = {}
        for m in m_values:
            print(f"Running {prior_name}, n={n}, m={m}...")
            A_est = monte_carlo_bayes(data, prior_params["nu0"], prior_params["delta0"], m)
            results[prior_name][n][m] = A_est

# Display results
for prior_name in results:
    print(f"\nResults using {prior_name}:")
    for n in results[prior_name]:
        print(f"n = {n}")
        for m in results[prior_name][n]:
            print(f"  m = {m}:")
            print(results[prior_name][n][m])
        print("-" * 40)

Running Prior A, n=10, m=1000...
Running Prior A, n=10, m=10000...
Running Prior A, n=10, m=100000...
Running Prior A, n=100, m=1000...
Running Prior A, n=100, m=10000...
Running Prior A, n=100, m=100000...
Running Prior A, n=1000, m=1000...
Running Prior A, n=1000, m=10000...
Running Prior A, n=1000, m=100000...
Running Prior B, n=10, m=1000...
Running Prior B, n=10, m=10000...
Running Prior B, n=10, m=100000...
Running Prior B, n=100, m=1000...
Running Prior B, n=100, m=10000...
Running Prior B, n=100, m=100000...
Running Prior B, n=1000, m=1000...
Running Prior B, n=1000, m=10000...
Running Prior B, n=1000, m=100000...

Results using Prior A:
n = 10
  m = 1000:
[[1.18309727 0.01104977]
 [0.01104977 1.57848646]]
  m = 10000:
[[1.17908785 0.02445063]
 [0.02445063 1.55894717]]
  m = 100000:
[[1.18884333 0.02559059]
 [0.02559059 1.57416565]]
----------------------------------------
n = 100
  m = 1000:
[[ 1.13657998 -0.01227224]
 [-0.01227224  1.97607822]]
  m = 10000:
[[1.06569515 0.015

In [ ]:
 import numpy as np
from scipy.stats import invwishart, invgamma

# --- Configuration ---
np.random.seed(42)
n = 100  # number of data points
d = 2    # dimension
nu = 3
A1, A2 = 0.05, 0.05
iterations = 1000

# True covariance
Sigma_true = np.array([[1, 0], [0, 2]])

# Generate data
Y = np.random.multivariate_normal(mean=[0, 0], cov=Sigma_true, size=n)

# Initialize a1, a2, Sigma
a = np.array([1.0, 1.0])
Sigma = np.eye(d)

# For storing Sigma samples
Sigma_samples = []

# Precompute Y^T Y (sum of y_i y_i^T)
YTY = Y.T @ Y

# --- Gibbs Sampling ---
for _ in range(iterations):
    # Step 1: Sample Sigma | Y, a
    scale_matrix = 2 * nu * np.diag(1 / a) + YTY
    df = nu + d + n - 1
    Sigma = invwishart.rvs(df=df, scale=scale_matrix)

    # Step 2: Sample a_k | Y, Sigma
    Sigma_inv = np.linalg.inv(Sigma)
    for k in range(d):
        shape = (nu + n) / 2
        rate = nu * Sigma_inv[k, k] + 1 / (A1**2 if k == 0 else A2**2)
        a[k] = invgamma.rvs(a=shape, scale=rate)

    Sigma_samples.append(Sigma)

# Final estimate: posterior mean
Sigma_est = np.mean(Sigma_samples, axis=0)

print("Posterior covariance estimate after 1000 Gibbs iterations:")
print(np.round(Sigma_est, 4))

Posterior covariance estimate after 1000 Gibbs iterations:
[[0.9833 0.0363]
 [0.0363 1.4816]]


In [ ]:
 import numpy as np
from scipy.stats import invwishart, invgamma

# --- Configuration ---
np.random.seed(42)
n = 100  # number of data points
d = 2    # dimension
nu = 3
A1, A2 = 0.05, 0.05
iterations = 10000

# True covariance
Sigma_true = np.array([[1, 0], [0, 2]])

# Generate data
Y = np.random.multivariate_normal(mean=[0, 0], cov=Sigma_true, size=n)

# Initialize a1, a2, Sigma
a = np.array([1.0, 1.0])
Sigma = np.eye(d)

# For storing Sigma samples
Sigma_samples = []

# Precompute Y^T Y (sum of y_i y_i^T)
YTY = Y.T @ Y

# --- Gibbs Sampling ---
for _ in range(iterations):
    # Step 1: Sample Sigma | Y, a
    scale_matrix = 2 * nu * np.diag(1 / a) + YTY
    df = nu + d + n - 1
    Sigma = invwishart.rvs(df=df, scale=scale_matrix)

    # Step 2: Sample a_k | Y, Sigma
    Sigma_inv = np.linalg.inv(Sigma)
    for k in range(d):
        shape = (nu + n) / 2
        rate = nu * Sigma_inv[k, k] + 1 / (A1**2 if k == 0 else A2**2)
        a[k] = invgamma.rvs(a=shape, scale=rate)

    Sigma_samples.append(Sigma)

# Final estimate: posterior mean
Sigma_est = np.mean(Sigma_samples, axis=0)

print("Posterior covariance estimate after 1000 Gibbs iterations:")
print(np.round(Sigma_est, 4))

Posterior covariance estimate after 1000 Gibbs iterations:
[[0.9869 0.0338]
 [0.0338 1.4719]]


In [ ]:
import numpy as np
from scipy.special import gammaln
from scipy.optimize import minimize_scalar
from scipy.stats import invwishart

# Sample generation (assumed)
np.random.seed(42)
true_cov = np.array([[1, 0], [0, 2]])
n = 1000
d = 2
y = np.random.multivariate_normal(np.zeros(d), true_cov, size=n)

# Sample covariance sum
S = sum(np.outer(yi, yi) for yi in y)

# Empirical Bayes: Define log marginal likelihood (unnormalized)
def log_marginal_likelihood(nu, S, n, d):
    term1 = nu * np.log((nu + n) / nu)
    term2 = n * np.log((nu + n) / n)

    def log_multigamma(a, p):
        return (p * (p - 1) / 4) * np.log(np.pi) + np.sum(gammaln(a - (np.arange(p) / 2)))

    term3 = log_multigamma(nu / 2, d)
    term4 = log_multigamma((nu + n) / 2, d)

    return -(term1 + term2 + term3 - term4)

# Optimize ν using scalar minimization
res = minimize_scalar(lambda nu: -log_marginal_likelihood(nu, S, n, d),
                      bounds=(d + 2, 500), method='bounded')
nu_opt = res.x

# Compute ∆opt
Delta_opt = (nu_opt / n) * S

# Compute posterior mean of Inverse Wishart
posterior_df = nu_opt + n
posterior_scale = Delta_opt + S
posterior_mean = posterior_scale / (posterior_df - d - 1)

# Print results
print("Optimal ν:", nu_opt)
print("Posterior covariance estimate (Empirical Bayes):")
print(np.round(posterior_mean, 4))

Optimal ν: 499.9999867106493
Posterior covariance estimate (Empirical Bayes):
[[1.0344 0.0063]
 [0.0063 1.8534]]
